## API Testing

In [1]:
import requests, json

In [ ]:


url = "https://fastapi-app-613987678533.asia-southeast1.run.app/upload-pill-image/"
files = {"file": open("test_pills/Bewell-C.jpg", "rb")}

response = requests.post(url, files=files)
response.json()

In [ ]:
import requests, json

url = "http://192.168.254.118:8000/upload-pill-image/"
files = {"file": open("test_pills/Bewell-C.jpg", "rb")}

response = requests.post(url, files=files)
response.json()

## Testing

In [30]:
import medication_matching
import ocr_module

[2025/06/23 13:22:27] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Jandrik/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_

In [31]:
image_path = 'test_images/test2.jpg'

In [38]:
import package_identification

package_identification.get_info(image_path)

[2025/06/23 13:24:49] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.10302162170410156
[2025/06/23 13:24:49] ppocr DEBUG: cls num  : 10, elapsed : 0.023677349090576172
[2025/06/23 13:24:50] ppocr DEBUG: rec_res num  : 10, elapsed : 0.585432767868042


{'matches': [{'INDEX': 6319,
   'Registration Number': 'DRP-8154',
   'Generic Name': 'Levothyroxine Sodium',
   'Brand Name': 'Euthyrox 50',
   'Dosage Strength': '50 mcg',
   'Dosage Form': 'Tablet',
   'Classification': 'Prescription Drug (RX)',
   'Packaging': "Alu-Alu Blister Pack x 25s (Box of 100's)",
   'Pharmacologic Category': 'Thyroid Hormone Replacement',
   'Manufacturer': 'Merck S.A. DE C.V.',
   'Country of Origin': 'Mexico',
   'Trader': nan,
   'Importer': 'Merck, Inc.',
   'Distributor': nan,
   'Application Type': 'Renewal',
   'Issuance Date': '23-Jun-24',
   'Expiry Date': '14-Aug-29',
   'match_score': 79.1,
   'rx_info': {'Name': 'Levothyroxine Sodium (Levoxyl)',
    'URL': 'https://www.rxlist.com/levoxyl-drug.htm',
    'What is': '\nLevoxyl (levothyroxine sodium) Tablets is a replacement for a hormone that is normally produced by your thyroid gland used to treat hypothyroidism (low thyroid hormone). Levoxyl is also used to treat or prevent goiter (enlarged thyro

In [32]:
# Perform OCR on the downloaded image
ocr_text = ocr_module.extract_image_text(image_path=image_path)
# Get medication info
med_info = medication_matching.get_info(ocr_text, threshold=60)

[2025/06/23 13:22:29] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.4695017337799072
[2025/06/23 13:22:29] ppocr DEBUG: cls num  : 10, elapsed : 0.12003445625305176
[2025/06/23 13:22:30] ppocr DEBUG: rec_res num  : 10, elapsed : 1.1342177391052246


In [33]:
from fuzzywuzzy import fuzz
import os

available_package_images_names = os.listdir('MedsForAll_Images')


In [36]:
image_matches = []
threshold = 60

for match in med_info['matches']:
    match_name = f"{match['Generic Name']} {match['Brand Name']} {match['Dosage Strength']}"
    best_image_match = None
    best_match_score = 0
    for package_name in available_package_images_names:
        
        score = fuzz.token_sort_ratio(match_name, package_name[:-3].replace('-', '').strip())
        if score > best_match_score:
            best_match_score = score
            best_image_match = package_name

    if best_match_score >= threshold:
        image_matches.append(f'/MedsForAll_Images/{best_image_match}')
    else:
        image_matches.append('')

In [37]:
image_matches

['/MedsForAll_Images/--LEVOTHYROXINE SODIUM ----EUTHYROX 50MCG TABLET----50mcg Tablet----.jpg',
 '/MedsForAll_Images/--LEVOTHYROXINE SODIUM ----EUTHYROX 50MCG TABLET----50mcg Tablet----.jpg',
 '/MedsForAll_Images/--LEVOTHYROXINE SODIUM ----EUTHYROX 50MCG TABLET----50mcg Tablet----.jpg']

In [2]:
import pickle

In [4]:
with open('pill_model/traintestsplit.pkl', 'rb') as tts_pkl:
    traintestsplit = pickle.load(tts_pkl)

In [7]:
pill_names_list = list(traintestsplit['class_to_label'].keys())

In [ ]:
from pill_identification import preprocess_image, predict, get_info
import pill_identification
import os

In [25]:
reload(pill_identification)

Model Loadded.


c:\Users\Jandrik\Documents\MyFILES\Programming\Python Files\Programs\vino\PillandPackageIdentification\pill_identification.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

<module 'pill_identification' from 'c:\\Users\\Jandrik\\Documents\\MyFILES\\Programming\\Python Files\\Programs\\vino\\PillandPackageIdentification\\pill_identification.py'>

In [26]:
image_path = 'test_pills/Bewell-C.jpg'


info = pill_identification.get_info(image_path)
# image = pill_identification.preprocess_image(image_path)
# predictions, scores = pill_identification.predict(image)



In [27]:
info

{'matches': [{'INDEX': 2351,
   'Registration Number': 'DRHR-1355',
   'Generic Name': 'Ascorbic Acid',
   'Brand Name': 'Bewell C',
   'Dosage Strength': '500 mg',
   'Dosage Form': 'Capsule',
   'Classification': 'Household Remedy (HR)',
   'Packaging': "Alu-Red PVC Blister Pack x 10's (Box of 100's and 200's)",
   'Pharmacologic Category': '-',
   'Manufacturer': 'Lejal Laboratories Inc.',
   'Country of Origin': 'Philippines',
   'Trader': 'Bewell Nutraceutical Corp.',
   'Importer': nan,
   'Distributor': nan,
   'Application Type': '-',
   'Issuance Date': '24-Jul-20',
   'Expiry Date': '28-Aug-25',
   'Score': 0.85,
   'rx_info': {'Name': 'Ascorbic Acid (Vitamin C)',
    'URL': 'https://www.rxlist.com/ascorbic-acid-drug.htm',
    'What is': '\nAscorbic Acid (vitamin C) is a water-soluble vitamin recommended for the prevention and treatment of scurvy. Ascorbic acid is available in generic form.',
    'What Are Side Effects': '\nCommon side effects of ascorbic acid include transie